# AlexNet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# convolution, fully_connected

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        # in, out_channels가 이미지 사이즈가 아니라 채널 사이즈(filter, feature 개수)를 가지고 가는거구나
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4)
        self.conv_layer2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.conv_layer3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv_layer4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv_layer5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc_layer1 = nn.Linear(in_features=6*6*256, out_features=4096)
        self.fc_layer2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc_layer3 = nn.Linear(in_features=4096, out_features=1000)
    
    # 이건 alexnet을 구현한 git에서 가져온 code
    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.net[4].bias, 1)
        nn.init.constant_(self.net[10].bias, 1)
        nn.init.constant_(self.net[12].bias, 1)

    # x1, x2 = 병렬 GPU에 각각 처리되는 것을 구현하기 위한 변수
    def forward(self, x):
        # Layer_1
        # ReLU
        
        x = F.relu(self.conv_layer1(x))
        # LRN (LocalResponseNormalization, 정규화)
        x = F.local_response_norm(x, size=5, alpha=0.0001, beta=0.75, k=2)
        # MaxPooling
        x = F.max_pool2d(x, kernel_size=3, stride=2)

        # Layer_2
        x = F.relu(self.conv_layer2(x))
        x = F.local_response_norm(x, size=5, alpha=0.0001, beta=0.75, k=2)
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        
        # Layer_3
        x = F.relu(self.conv_layer3(x))
        
        # Layer_4
        x = F.relu(self.conv_layer4(x))
        
        # Layer_5
        x = F.relu(self.conv_layer5(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        
        # Flatten
        x = nn.Flatten(x)
        # x = x.flatten()
        # x = x.view(-1, 6*6*256)

        # Layer_6
        x = nn.Dropout(x, p=0.5)
        x = F.relu(self.fc_layer1(x))

        # Layer_7
        x = nn.Dropout(x, p=0.5)
        x = F.relu(self.fc_layer2(x))

        # Layer_8
        x = nn.Softmax(self.fc_layer3(x))

        return x

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [15]:
alexnet = AlexNet().to(device)
print(alexnet)

AlexNet(
  (conv_layer1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv_layer2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv_layer3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_layer5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc_layer1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc_layer2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc_layer3): Linear(in_features=4096, out_features=1000, bias=True)
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=alexnet.parameters(), lr=1e-4)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=30, gamma=0.1)
